In [8]:
import pandas as pd
import numpy as np

In [9]:
def cleanStrings(row, column):
    return str(int(float(row[column])))

In [10]:
# df_e = pd.read_csv('data/finalized/enrollment.csv')
df_p = pd.read_csv('data/finalized/proficiency_finalized.csv')

In [11]:
print(df_p.columns)

Index(['Unnamed: 0', 'above_city_avg', 'above_st_avg', 'charter', 'city_avg',
       'city_avg_imp', 'city_gap_close', 'city_imp', 'city_quin',
       'city_quin_imp', 'city_std', 'city_std_imp', 'city_z', 'city_z_imp',
       'city_z_prev', 'district', 'district_id', 'grade',
       'grant_flag_district', 'grant_flag_school', 'group', 'in_city',
       'pct_at_level', 'school', 'school_id', 'st_avg', 'st_avg_imp',
       'st_gap_close', 'st_imp', 'st_quin', 'st_quin_imp', 'st_std',
       'st_std_imp', 'st_z', 'st_z_imp', 'st_z_prev', 'state', 'subject',
       'top_20_city', 'top_20_st', 'year', 'start_sy', 'start_sy_d',
       'start_sy_s', 'edchamp_id', 'edchamp_name', 'city', 'latitude',
       'longitude', 'first_grant_start', 'launch_year_spring',
       'grants_district_growth', 'grants_district_newstart',
       'grants_district_qa', 'grants_school_growth', 'grants_school_newstart',
       'grants_school_qa', 'max', 'min', 'grade5flag', 'grade8flag',
       'grade_config'],
  

In [12]:
temp = df_p.groupby(['state', 'year', 'district_id', 'school_id', 'subject', 'grade', 'group'], as_index=False)['district'].count()
print(temp[temp['district'] > 1].head())
print(temp[temp['district'] > 1].shape)

Empty DataFrame
Columns: [state, year, district_id, school_id, subject, grade, group, district]
Index: []
(0, 8)


In [13]:
temp = temp.rename(columns={'district': 'count'})
temp2 = df_p.merge(temp, on=['state', 'year', 'district_id', 'school_id', 'subject', 'grade', 'group'])
temp2[temp2['count'] > 1].to_csv('temp.csv')

In [3]:
print(df_p['city_quin'].drop_duplicates())

0          1.0
2          3.0
3          2.0
95         5.0
117        4.0
1327741    NaN
Name: city_quin, dtype: float64


In [7]:
# import grant data
df_grant = pd.read_csv('data/finalized/grants_clean_unifiedForAnalysis.csv',dtype={'grantmerge': str,'state':str,'educationchampion':str,'grantid':str,'grantscope':str,'district_id':str,'stateleaname':str,'previousstateleaid':str,'schoolgranteeid':str,'school_id':str,'stateschoolname':str,'previousstateschoolid':str,'granteeisthesameasgrantben':str,'granteenameifdifferentfrom':str,'leagrantstrategy':str,'strategyenrollmentmax':str,'strategyqualityimprovement':str,'strategyexpansion':str,'strategynewstart':str,'endofgrantleaenrollmentgoal':str,'granttype':str,'supportorganizationsinvolved':str,'levelofedchampinvolvement':str,'duration':str,'startdate':str,'enddate':str,'totalamountcommitted':np.float64,'totalamountdisbursed':np.float64,'otherfundscontributed':str,'notes':str,'schooltype':str,'schoolgrantstrategy':str,'endofgrantenrollmentgoal':np.float64,'wasanyportionofthetotalgra':str,'portionedamountcommitted':np.float64,'portionedamountdisbursed':np.float64,'no_id_status':str})

# apply cleanStrings function to district_id and school_id
df_grant['district_id'] = df_grant.apply(lambda x: cleanStrings(x, 'district_id') ,axis=1)
df_grant['school_id'] = df_grant.apply(lambda x: cleanStrings(x, 'school_id') ,axis=1)
grants_district = df_grant[df_grant.grantscope.eq('LEA-wide')]
grants_school = df_grant[df_grant.grantscope.eq('School-based')]

In [9]:
print(grants_district.groupby(['educationchampion'])['totalamountcommitted'].sum())

educationchampion
Boston Schools Fund                  1744810.00
City Education Partners             10213266.00
Educate 78                            278025.00
Great MN Schools                     9511064.15
New Jersey Children's Foundation      780000.00
RedefineEd                           2975000.00
SchoolSmart KC                      10973193.00
Name: totalamountcommitted, dtype: float64


In [18]:
temp = df_e.dropna(subset=['grantid', 'grantid_grant_school'], how='all')
grouped_by = ['state', 'edchamp_name', 'year', 'district_id', 'school_id', 'in_city', 'grade', 'group']
temp = temp.groupby(grouped_by, as_index=False)['num'].min()
print(temp.head(5))
temp = temp[temp.year.eq(2019) & temp.grade.eq('All') & temp.group.eq('All Students') & temp.in_city.eq(1)]
print(temp.head(5))
print(temp.groupby(['edchamp_name', 'year'])['num'].sum())

  state edchamp_name  year district_id school_id  in_city grade         group  \
0    ca   Educate 78  2015       10017    112607      0.0   All  All Students   
1    ca   Educate 78  2015       10017    112607      0.0   All         Black   
2    ca   Educate 78  2015       10017    112607      0.0   All      Hispanic   
3    ca   Educate 78  2015       10017    112607      0.0   All    Low-Income   
4    ca   Educate 78  2015       10017    112607      0.0   All         White   

   num  
0  395  
1  163  
2  185  
3  186  
4   22  
     state edchamp_name  year district_id school_id  in_city grade  \
8688    ca   Educate 78  2019       61259    100065      1.0   All   
8694    ca   Educate 78  2019       61259    100701      1.0   All   
8717    ca   Educate 78  2019       61259    100792      1.0   All   
8736    ca   Educate 78  2019       61259    102988      1.0   All   
8741    ca   Educate 78  2019       61259    106542      1.0   All   

             group  num  
8688  All St

In [6]:
temp = df_p[df_p.state.eq('mn')]
print(temp[['year', 'grade', 'group']].drop_duplicates())

         year grade         group
2663219  2013   All  All Students
2667287  2013   All    Low-Income
2671152  2013   All         White
2673992  2013   All         Black
2676438  2013   All      Hispanic
2708100  2014   All  All Students
2712537  2014   All    Low-Income
2716492  2014   All         White
2719274  2014   All         Black
2721630  2014   All      Hispanic
2754426  2015   All  All Students
2758948  2015   All    Low-Income
2763042  2015   All         White
2765858  2015   All         Black
2768392  2015   All      Hispanic
2801702  2016   All  All Students
2806034  2016   All    Low-Income
2810019  2016   All         White
2812908  2016   All         Black
2815377  2016   All      Hispanic
2848247  2017   All  All Students
2852638  2017   All    Low-Income
2856620  2017   All         White
2859465  2017   All         Black
2861935  2017   All      Hispanic
2895168  2018   All  All Students
2899420  2018   All    Low-Income
2903337  2018   All         White
2906156  2018 